In [1]:
print("hello")

hello


In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TestSpark").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 10:06:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv("/Users/DELL/Downloads/dataset.csv", header=True, inferSchema=True)

In [5]:
df.head(5)

[Row(Date='10-Sep-12', Time (Local)=7, Eyeballs =5, Zeroes =0, Completed Trips =2, Requests =2, Unique Drivers=9),
 Row(Date=None, Time (Local)=8, Eyeballs =6, Zeroes =0, Completed Trips =2, Requests =2, Unique Drivers=14),
 Row(Date=None, Time (Local)=9, Eyeballs =8, Zeroes =3, Completed Trips =0, Requests =0, Unique Drivers=14),
 Row(Date=None, Time (Local)=10, Eyeballs =9, Zeroes =2, Completed Trips =0, Requests =1, Unique Drivers=14),
 Row(Date=None, Time (Local)=11, Eyeballs =11, Zeroes =1, Completed Trips =4, Requests =4, Unique Drivers=11)]

In [7]:
#using spark api
completed_trips_by_date = df.groupBy("Date").sum("Completed Trips ")

In [9]:
completed_trips_by_date

DataFrame[Date: string, sum(Completed Trips ): bigint]

In [17]:
from pyspark.sql.functions import sum as _sum, desc

# Group by and compute total
completed_trips_by_date = df.groupBy("Date").agg(_sum("Completed Trips").alias("TotalTrips"))

# Find date with most completed trips
date_with_most_completed_trips = completed_trips_by_date \
    .orderBy(desc("TotalTrips")) \
    .select("Date") \
    .first()["Date"]

print("📅 Date with most completed trips:", date_with_most_completed_trips)


📅 Date with most completed trips: None


In [22]:
from pyspark.sql.functions import col, sum as _sum, desc

# Strip spaces from column names
uber = uber.select([col(c).alias(c.strip()) for c in uber.columns])

# Drop rows where Date is null
uber = uber.filter(col("Date").isNotNull())

# Ensure 'Completed Trips' is integer
uber = uber.withColumn("Completed Trips", col("Completed Trips").cast("int"))

# Group and find total trips
completed_trips_by_date = uber.groupBy("Date").agg(_sum("Completed Trips").alias("TotalTrips"))

# Order by total trips
completed_trips_by_date.orderBy(desc("TotalTrips")).show()

# Find date with most trips
top_row = completed_trips_by_date.orderBy(desc("TotalTrips")).first()
if top_row:
    print("📅 Date with most completed trips:", top_row["Date"], "🚗 Trips:", top_row["TotalTrips"])
else:
    print("❌ No valid data found.")


+---------+----------+
|     Date|TotalTrips|
+---------+----------+
|15-Sep-12|        23|
|23-Sep-12|        23|
|22-Sep-12|        19|
|16-Sep-12|        17|
|21-Sep-12|        10|
|18-Sep-12|         3|
|14-Sep-12|         3|
|20-Sep-12|         2|
|10-Sep-12|         2|
|11-Sep-12|         1|
|24-Sep-12|         1|
|13-Sep-12|         0|
|17-Sep-12|         0|
|12-Sep-12|         0|
|19-Sep-12|         0|
+---------+----------+

📅 Date with most completed trips: 15-Sep-12 🚗 Trips: 23


In [25]:
uber.head(5) #uber has the grouped date 

[Row(Date='10-Sep-12', Time (Local)=7, Eyeballs=5, Zeroes=0, Completed Trips=2, Requests=2, Unique Drivers=9),
 Row(Date='11-Sep-12', Time (Local)=0, Eyeballs=9, Zeroes=3, Completed Trips=1, Requests=1, Unique Drivers=3),
 Row(Date='12-Sep-12', Time (Local)=0, Eyeballs=9, Zeroes=2, Completed Trips=0, Requests=1, Unique Drivers=2),
 Row(Date='13-Sep-12', Time (Local)=0, Eyeballs=11, Zeroes=11, Completed Trips=0, Requests=2, Unique Drivers=0),
 Row(Date='14-Sep-12', Time (Local)=0, Eyeballs=10, Zeroes=1, Completed Trips=3, Requests=4, Unique Drivers=3)]

In [31]:
df.head(5)

[Row(Date='10-Sep-12', Time (Local)=7, Eyeballs=5, Zeroes=0, Completed Trips=2, Requests=2, Unique Drivers=9),
 Row(Date=None, Time (Local)=8, Eyeballs=6, Zeroes=0, Completed Trips=2, Requests=2, Unique Drivers=14),
 Row(Date=None, Time (Local)=9, Eyeballs=8, Zeroes=3, Completed Trips=0, Requests=0, Unique Drivers=14),
 Row(Date=None, Time (Local)=10, Eyeballs=9, Zeroes=2, Completed Trips=0, Requests=1, Unique Drivers=14),
 Row(Date=None, Time (Local)=11, Eyeballs=11, Zeroes=1, Completed Trips=4, Requests=4, Unique Drivers=11)]

In [34]:
from pyspark.sql.functions import concat_ws, lpad, to_timestamp, sum, window, lit


# Step 1: Combine Date and Time into a proper datetime string
df = df.withColumn(
    "datetime_string",
    concat_ws(" ", "Date", concat_ws(":", lpad("Time (Local)", 2, "0"), lpad(lit("00"), 2, "0")))
)

# Step 2: Convert to timestamp (handle format like '15-Sep-12 16:00')
df = df.withColumn("datetime", to_timestamp("datetime_string", "dd-MMM-yy HH:mm"))

# Step 3: Group by 24-hour window and sum the completed trips
completed_trips_by_window = df \
    .groupBy(window("datetime", "24 hours")) \
    .agg(sum("Completed Trips").alias("Total Completed Trips")) \
    .orderBy("Total Completed Trips", ascending=False)

# Step 4: Get the highest number of completed trips in a 24-hour window
highest_completed_trips_in_24_hours = completed_trips_by_window \
    .select("Total Completed Trips") \
    .first()["Total Completed Trips"]

print("🚗 Highest completed trips in any 24-hour window:", highest_completed_trips_in_24_hours)


25/06/11 10:32:30 ERROR Executor: Exception in task 0.0 in stage 44.0 (TID 35)
org.apache.spark.SparkDateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '08:00' could not be parsed at index 2. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007
	at org.apache.spark.sql.errors.QueryExecutionErrors$.ansiDateTimeParseError(QueryExecutionErrors.scala:279)
	at org.apache.spark.sql.errors.QueryExecutionErrors.ansiDateTimeParseError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvalu

DateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '08:00' could not be parsed at index 2. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

In [35]:
from pyspark.sql.functions import hour, sum

hourly_requests = df.groupBy(hour("Time (Local)").alias("hour")).agg(sum("Requests").alias("total_requests")).orderBy("total_requests", ascending=False)

most_requested_hour = hourly_requests.select("hour").first()[0]
print("The hour with the most requests is:", most_requested_hour)


{"ts": "2025-06-11 10:33:16.128", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve \"hour(Time (Local))\" due to data type mismatch: The first parameter requires the \"(TIMESTAMP OR TIMESTAMP WITHOUT TIME ZONE)\" type, however \"Time (Local)\" has the type \"INT\". SQLSTATE: 42K09", "context": {"file": "java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)", "line": "", "fragment": "hour", "errorClass": "DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o695.agg.\n: org.apache.spark.sql.AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve \"hour(Time (Local))\" due to data type mismatch: The first parameter requires the \"(TIMESTAMP OR TIMESTAMP WITHOUT TIME ZONE)\" type, however \"Time (Local)\" has the type \"INT\". SQLSTATE: 42K09;\n'Aggregate [hour(Time (Local)#42, Some(Asia/Kolkata))], [h

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "hour(Time (Local))" due to data type mismatch: The first parameter requires the "(TIMESTAMP OR TIMESTAMP WITHOUT TIME ZONE)" type, however "Time (Local)" has the type "INT". SQLSTATE: 42K09;
'Aggregate [hour(Time (Local)#42, Some(Asia/Kolkata))], [hour(Time (Local)#42, Some(Asia/Kolkata)) AS hour#428, sum(Requests#46) AS total_requests#429L]
+- Project [Date#41, Time (Local)#42, Eyeballs#43, Zeroes#44, Completed Trips#45, Requests#46, Unique Drivers#47, datetime_string#407, to_timestamp(datetime_string#407, Some(dd-MMM-yy HH:mm), TimestampType, Some(Asia/Kolkata), true) AS datetime#408]
   +- Project [Date#41, Time (Local)#42, Eyeballs#43, Zeroes#44, Completed Trips#45, Requests#46, Unique Drivers#47, concat_ws( , Date#41, concat_ws(:, lpad(cast(Time (Local)#42 as string), 2, 0), lpad(00, 2, 0))) AS datetime_string#407, datetime#387]
      +- Project [Date#41, Time (Local)#42, Eyeballs#43, Zeroes#44, Completed

In [36]:
#3. Which hour of the day had the most requests during the two-week period?
df.groupBy("Time (Local)") \
    .agg(sum("Requests").alias("TotalRequests")) \
    .orderBy(desc("TotalRequests")) \
    .show(1)



+------------+-------------+
|Time (Local)|TotalRequests|
+------------+-------------+
|          23|          184|
+------------+-------------+
only showing top 1 row


In [37]:
#4What percentage of all zeroes occurred on the weekend (Fri 5pm to Sun 3am)?
from pyspark.sql.functions import hour, to_timestamp, date_format

df = df.withColumn("Timestamp", to_timestamp("Date", "dd-MMM-yy")) \
       .withColumn("Weekday", date_format("Timestamp", "E")) \
       .withColumn("Hour", df["Time (Local)"].cast("int"))

# Weekend filter: Fri 17:00 - Sun 03:00
weekend_df = df.filter(
    ((df["Weekday"] == "Fri") & (df["Hour"] >= 17)) |
    (df["Weekday"] == "Sat") |
    ((df["Weekday"] == "Sun") & (df["Hour"] < 3))
)

# Sum Zeroes
from pyspark.sql.functions import sum

total_zeroes = df.agg(sum("Zeroes")).collect()[0][0]
weekend_zeroes = weekend_df.agg(sum("Zeroes")).collect()[0][0]

percentage = (weekend_zeroes / total_zeroes) * 100
print(f"📊 Weekend Zeroes: {percentage:.2f}%")


📊 Weekend Zeroes: 2.31%


In [39]:
#Q5.What is the weighted average ratio of completed trips per driver during the two-week period? Tip: “Weighted average” means your answer should account for the total trip volume in each hour to determine the most accurate number in the whole period.
from pyspark.sql.functions import col, expr, sum

# Safe division: try_divide avoids division-by-zero error
df = df.withColumn("TripDriverRatio", expr("try_divide(`Completed Trips`, `Unique Drivers`)"))

# Weighted average = sum(ratio * completed trips) / sum(completed trips)
df = df.withColumn("Weight", col("TripDriverRatio") * col("Completed Trips"))

weighted_avg = df.agg((sum("Weight") / sum("Completed Trips")).alias("WeightedAvg")) \
                 .first()["WeightedAvg"]

print(f"⚖ Weighted average: {weighted_avg:.2f} trips per driver")


⚖ Weighted average: 0.83 trips per driver


In [40]:
#Q6.In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two-week period in terms of unique requests? A new shift starts every 8 hours. Assume that a driver will work the same shift each day.
from pyspark.sql.functions import when

df = df.withColumn("Shift", when(col("Time (Local)") < 8, "Shift_1")
                              .when(col("Time (Local)") < 16, "Shift_2")
                              .otherwise("Shift_3"))

df_shift = df.groupBy("Shift").agg(sum("Requests").alias("TotalRequests")) \
             .orderBy(desc("TotalRequests"))

df_shift.show()


+-------+-------------+
|  Shift|TotalRequests|
+-------+-------------+
|Shift_3|         1032|
|Shift_1|          446|
|Shift_2|          380|
+-------+-------------+



In [41]:
#Q7.True or False: Driver supply always increases when demand increases during the two-week period. Tip: Visualize the data to confirm your answer if needed.
from pyspark.sql.functions import corr

correlation = df.select(corr("Requests", "Unique Drivers")).first()[0]
print(f"📈 Correlation: {correlation:.2f}")
print("✅ True" if correlation > 0.5 else "❌ False")


📈 Correlation: 0.69
✅ True


In [42]:
#Q8.In which 72-hour period is the ratio of Zeroes to Eyeballs the highest?
from pyspark.sql.window import Window

# Window over 3 days
df = df.withColumn("EyeballsInt", col("Eyeballs").cast("int")) \
       .withColumn("ZeroesInt", col("Zeroes").cast("int"))

windowSpec = Window.orderBy("Date").rowsBetween(0, 71)

df = df.withColumn("SumZeroes", sum("ZeroesInt").over(windowSpec)) \
       .withColumn("SumEyeballs", sum("EyeballsInt").over(windowSpec)) \
       .withColumn("Ratio", col("SumZeroes") / col("SumEyeballs")) \
       .orderBy(desc("Ratio"))

df.select("Date", "Ratio").show(1)


25/06/11 11:07:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/11 11:07:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/11 11:07:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/11 11:07:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/11 11:07:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+-----------------+
|     Date|            Ratio|
+---------+-----------------+
|23-Sep-12|0.391304347826087|
+---------+-----------------+
only showing top 1 row


In [43]:
#9. If you could add 5 drivers to a single hour daily, which one?
df.groupBy("Time (Local)") \
    .agg(
        sum("Eyeballs").alias("TotalEyeballs"),
        sum("Unique Drivers").alias("TotalDrivers")
    ) \
    .withColumn("EyeballPerDriver", col("TotalEyeballs") / col("TotalDrivers")) \
    .orderBy(desc("EyeballPerDriver")) \
    .show(1)


+------------+-------------+------------+-----------------+
|Time (Local)|TotalEyeballs|TotalDrivers| EyeballPerDriver|
+------------+-------------+------------+-----------------+
|          23|          488|         119|4.100840336134453|
+------------+-------------+------------+-----------------+
only showing top 1 row


In [45]:
from pyspark.sql.functions import sum, col, desc

# Group by hour of day (0-23) and calculate total eyeballs and total drivers
hourly_demand = df.groupBy("Time (Local)").agg(
    sum("Eyeballs").alias("TotalEyeballs"),
    sum("Unique Drivers").alias("TotalDrivers")
)

# Calculate pressure score = Eyeballs per driver, safely avoiding division by zero
hourly_demand = hourly_demand.withColumn(
    "PressureScore",
    expr("try_divide(TotalEyeballs, TotalDrivers)")
)

# Find the hour with the highest pressure score
busiest_hour = hourly_demand.orderBy(desc("PressureScore")).first()

print(f"⏰ Best hour to add drivers: {busiest_hour['Time (Local)']}h")
print(f"🧮 Pressure score: {busiest_hour['PressureScore']:.2f}")


⏰ Best hour to add drivers: 23h
🧮 Pressure score: 4.10


In [47]:
#Q10.
df.groupBy("Time (Local)") \
    .agg(
        sum("Eyeballs").alias("TotalEyeballs"),
        sum("Requests").alias("TotalRequests"),
        sum("Unique Drivers").alias("TotalDrivers")
    ) \
    .withColumn("TotalActivity", col("TotalEyeballs") + col("TotalRequests") + col("TotalDrivers")) \
    .orderBy("TotalActivity") \
    .show(1)


+------------+-------------+-------------+------------+-------------+
|Time (Local)|TotalEyeballs|TotalRequests|TotalDrivers|TotalActivity|
+------------+-------------+-------------+------------+-------------+
|           4|           29|            9|           9|           47|
+------------+-------------+-------------+------------+-------------+
only showing top 1 row
